In [ ]:
from chi import server, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [ ]:
username = "project23"  # custom suffix for all resource names
username

In [ ]:
#username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-final-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium"  # specify the shared key uploaded to the project
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()


In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
security_groups = [
  {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
  {'name': "allow-5000", 'port': 5000, 'description': "Enable TCP port 5000 (used by MLFlow)"},
  {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by FastAPI)"},
  {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
  {'name': "allow-3000", 'port': 3000, 'description': "Enable TCP port 3000 (used by Grafana)"},
  {'name': "allow-9090", 'port': 9090, 'description': "Enable TCP port 9090 (used by Prometheus)"},
#  {'name': "allow-8080", 'port': 8080, 'description': "Enable TCP port 8080 (used by Airflow)"},
  {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by MinIO API)"},
  {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by MinIO Web UI)"},
]

In [ ]:
# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

for sg in security_groups:

  if not os_conn.get_security_group(sg['name']):
      os_conn.create_security_group(sg['name'], sg['description'])
      os_conn.create_security_group_rule(sg['name'], port_range_min=sg['port'], port_range_max=sg['port'], protocol='tcp', remote_ip_prefix='0.0.0.0/0')

  nova_server.add_security_group(sg['name'])

print(f"updated security groups: {[group.name for group in nova_server.list_security_group()]}")

In [ ]:
#s.execute("rm -rf restaurant-recommender && git clone https://github.com/srush-shah/restaurant-recommender")


In [ ]:
s.execute("git clone https://github.com/srush-shah/restaurant-recommender")

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
s.execute("curl https://rclone.org/install.sh | sudo bash")
s.execute("sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf")
s.execute("mkdir -p ~/.config/rclone")
s.execute('''cat > ~/.config/rclone/rclone.conf <<'EOF'
[chi_tacc]
type = swift
user_id = 59b49300deb6f11832f7764d9f2c3451ba51e578bb556d58e02bf4c64bd89a31
application_credential_id = aba2c756cb1649b6bd47c572f67a9528
application_credential_secret = HnUZC1Dvhnw1A5-19LiDQORiWURpL1IflqK3tk506NXJp8Tb0HSW4m0V9Ch4zEhwV9WR7tTxiAPTcTGHMi2SKw
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
EOF''')
s.execute("rclone lsd chi_tacc:")
s.execute("export RCLONE_CONTAINER=object-persist-project23")
s.execute("sudo mkdir -p /mnt/object")
s.execute("sudo chown -R cc /mnt/object")
s.execute("sudo chgrp -R cc /mnt/object")
s.execute("rclone mount chi_tacc:object-persist-project23 /mnt/object --read-only --allow-other --daemon")
s.execute("ls /mnt/object")

In [ ]:
s.execute("sh ~/restaurant-recommender/monitoring/start_services.sh")

In [ ]:
s.execute("
docker run -d --rm \
    -p 8888:8888 \
    -v /home/cc/my_workspace:/home/jovyan/work \
    -v /mnt/object:/home/jovyan/work/data \
    --name jupyter \
    quay.io/jupyter/pytorch-notebook:pytorch-2.5.1
")
